In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import Counter
from datetime import datetime

import geopandas as gpd
import shapely.geometry
import rasterio
import json
import geopandas as gpd
import geopandas_osm.osm
from descartes import PolygonPatch
import h5py 
from scipy.misc import imresize
import shapely.geometry
import tensorflow as tf
import cv2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
INPUT_SIZE = 240 

#helper functions:
def scale_bands(img, lower_pct = 1, upper_pct = 99):
    """Rescale the bands of a multichannel image for display"""
    img_scaled = np.zeros(img.shape, np.uint8)
    for i in range(img.shape[2]):
        band = img[:, :, i]
        lower, upper = np.percentile(band, [lower_pct, upper_pct])
        band = (band - lower) / (upper - lower) * 255
        img_scaled[:, :, i] = np.clip(band, 0, 255).astype(np.uint8)
    return img_scaled

def resize(image, new_shape):
    #img_resized = np.zeros(new_shape+(img.shape[2],)).astype('float32')
    #for i in range(img.shape[2]):
    #    img_resized[:, :, i] = imresize(img[:, :, i], new_shape, interp='bicubic')
    #img_resized = cv2.resize(img,dsize=(new_shape,new_shape))
    

    r = new_shape / (1.0*image.shape[1])
    dim = (new_shape, int(image.shape[0] * r))

    # perform the actual resizing of the image and show it
    img_resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    
    #make training masks the proper dimensionality of only 1-D
    if len(image.shape) == 2: img_resized = np.expand_dims(img_resized,axis=2)
    
    return img_resized

In [3]:
#identify a training/testing set

## make test/train split
#### assumes image and mask files are of identical number and none are missing

In [4]:
#image_files = !ls /home/ubuntu/data/TX_post/training_tiles_v1/*img.npy
#mask_files  = !ls /home/ubuntu/data/TX_post/training_tiles_v1/*mask.npy

#do the above without jupyter magic (!)
training_file_path = "/home/ubuntu/data/TX_post/training_tiles/"

ls = os.listdir(training_file_path)     #list isn't in order, need to sort the output

image_files = sorted([file for file in ls if file[-7:]=="img.npy"])  
mask_files = sorted([file for file in ls if file[-8:]=="mask.npy"])

def TT_file_split(image_files,mask_files,split=0.8):
    Xf_train, Yf_train, Xf_test, Yf_test = [],[],[],[]
    size = len(image_files)
    indexes = range(size)
    dice_roll = np.random.random(size)
    for i,roll in enumerate(dice_roll):
        if roll < split:  #add to train
            Xf_train.append(image_files[i])
            Yf_train.append( mask_files[i])
        if roll >= split: #add to test
            Xf_test.append(image_files[i])
            Yf_test.append( mask_files[i])
    return Xf_train, Yf_train, Xf_test, Yf_test

def TT_split(image_files,mask_files,split=0.8):
    X_train, Y_train, X_test, Y_test = [],[],[],[]
    size = len(image_files)
    indexes = range(size)
    dice_roll = np.random.random(size)
    for i,roll in enumerate(dice_roll):
        if roll < split:  #add to train
            X_train.append(np.load(image_files[i]))
            Y_train.append(np.load( mask_files[i]))
        if roll >= split: #add to test
            X_test.append(np.load(image_files[i]))
            Y_test.append(np.load( mask_files[i]))
    return np.array(X_train), np.array(Y_train), np.array(X_test), np.array(Y_test)

def TT_split_resize(image_files,mask_files,out_res=240,split=0.8):
    X_train, Y_train, X_test, Y_test = [],[],[],[]
    size = len(image_files)
    indexes = range(size)
    dice_roll = np.random.random(size)
    for i,roll in enumerate(dice_roll):
        if roll < split:  #add to train
            X_train.append(resize(np.load(image_files[i]),out_res))
            Y_train.append(resize(np.load(mask_files[i]),out_res))
        if roll >= split: #add to test
            X_test.append(resize(np.load(image_files[i]),out_res))
            Y_test.append(resize(np.load(mask_files[i]),out_res))
    return np.array(X_train), np.array(Y_train), np.array(X_test), np.array(Y_test)

## MAKE THE TESTING AND TRAINGING DATA SETS

In [ ]:
Xf_train, Yf_train, Xf_test, Yf_test  =\
            TT_file_split(image_files, mask_files, split=0.95)

In [ ]:
#write test and train file lists to disk
f = open('/home/ubuntu/Notebooks/test_train_filelists/test_images.txt', 'w')
for item in Xf_test:f.write("%s\n" % item)
f.close()
f = open('/home/ubuntu/Notebooks/test_train_filelists/test_masks.txt', 'w')
for item in Yf_test:f.write("%s\n" % item)
f.close() 
f = open('/home/ubuntu/Notebooks/test_train_filelists/train_images.txt', 'w')
for item in Xf_test:f.write("%s\n" % item)
f.close()
f = open('/home/ubuntu/Notebooks/test_train_filelists/train_masks.txt', 'w')
for item in Yf_test:f.write("%s\n" % item)
f.close()

In [6]:
X_train, Y_train, X_test, Y_test  = TT_split_resize(\
            image_files, mask_files, out_res = 240, split=0.95)

In [ ]:
#display a random training image
n = np.random.randint(0,len(X_train)-1) 

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,14))
ax1.imshow(X_train[n])
ax2.imshow(Y_train[n][:,:,0]);

In [8]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((4528, 240, 240, 3),
 (4528, 240, 240, 1),
 (258, 240, 240, 3),
 (258, 240, 240, 1))

In [9]:
#this must be a huge dataset... how big exactly?
import sys
sys.getsizeof(X_train)

782438544

In [10]:
# Source: https://github.com/jocicmarko/ultrasound-nerve-segmentation/blob/master/train.py

import keras
from keras import backend as K
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.models import Model
from keras.optimizers import Adam

# Set network size params
N_CLASSES = 1
N_CHANNEL = 3

# Define metrics
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Just put a negative sign in front of an accuracy metric to turn it into a loss to be minimized
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def jacc_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)

def jacc_coef_loss(y_true, y_pred):
    return -jacc_coef(y_true, y_pred)

def jacc_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)

def get_unet(lr=0.001):
    inputs = Input((INPUT_SIZE, INPUT_SIZE, N_CHANNEL))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv9)

    conv10 = Conv2D(N_CLASSES, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    # model.compile(optimizer=Adam(lr=lr), loss=jacc_coef_loss, metrics=[jacc_coef_int])
    model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=[jacc_coef_int])
    # model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=[dice_coef])

    return model


Using TensorFlow backend.


In [ ]:
X_train.shape,Y_train.shape

((4528, 240, 240, 3), (4528, 240, 240, 1))

In [ ]:
startTime = datetime.now()

from keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
# https://keras.io/callbacks/#reducelronplateau

# This sets the number of training epochs (you should do a lot more than 5)
NUM_EPOCHS = 1000

# Define callback to save model checkpoints
if not os.path.exists('checkpoints'):
    os.makedirs('checkpoints')
model_checkpoint = ModelCheckpoint(os.path.join('checkpoints', 'weights.{epoch:02d}-{val_loss:.5f}.hdf5'), monitor='loss', save_best_only=True)

# Define callback to reduce learning rate when learning stagnates
# This won't actually kick in with only 5 training epochs, but I'm assuming you'll train for hundreds of epochs when you get serious about training this NN.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, epsilon=0.002, cooldown=2)

# # Define rate scheduler callack (this is an alternative to ReduceLROnPlateau. There is no reason to use both.)
# schedule = lambda epoch_i: 0.01*np.power(0.97, i)
# schedule_lr = LearningRateScheduler(schedule)

# TensorBoard visuluaziations... this stuff is so freaking cool
tensorboard = TensorBoard(log_dir='/tmp/tboard_logs2', histogram_freq=0, write_graph=True, write_images=True)

# Train the model
model = get_unet(0.001)
model.fit(X_train, Y_train, batch_size=25, epochs=NUM_EPOCHS, verbose=1, shuffle=True, callbacks=[model_checkpoint, reduce_lr, tensorboard], validation_data=(X_test, Y_test))

print ("total runtime = ",datetime.now() - startTime)

Train on 4528 samples, validate on 258 samples
Epoch 1/1000
4528/4528 [==============================] - 567s - loss: -876.2753 - jacc_coef_int: 55.7741 - val_loss: -726.5957 - val_jacc_coef_int: 46.5674
Epoch 2/1000
1175/4528 [======>.......................] - ETA: 396s - loss: -894.4341 - jacc_coef_int: 57.0948